<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>CET Template Notebook</b> <br>
Contact author(s): <i>Author Name</i> <br>
Last verified to run: <i>yyyy-mm-dd</i> <br>
LSST Science Piplines version: Weekly <i>yyyy_xx</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>beginner</i> <br>

_In this template, text in italics are examples or instructions that should be: (a) removed if it is not applicable to the notebook; or (b) replaced with text that is appropriate for the notebook. But bold or regular text should appear pretty much as-is in all CET notebooks. For more information, see the [CET's Guidelines for Tutorial Notebooks](https://confluence.lsstcorp.org/pages/viewpage.action?pageId=168857070)._

_While developing, use the following code cell to check that the code conforms to standards, but then delete the cell and "Kernel --> Restart Kernel and Clear All Outputs" before saving and committing._

In [48]:
#%load_ext pycodestyle_magic
#%flake8_on
#import logging
#logging.getLogger("flake8").setLevel(logging.FATAL)

_The six cells below are considered the extended header of the notebook. The first four will be used, verbatim, to create the table of notebook metadata in the README.md file for the repository._

**Description:** _Very brief description of notebook._

**Skills:** _Brief list of skills to match the README.md file for the repository._

**LSST Data Products:** _List the all of the types of LSST catalogs and images used._

**Packages:** _List the python packages used._ (_List the packages being taught first, e.g., afwDisplay for a notebook about displaying images. Then supporting packages, e.g., lsst.daf.butler for a notebook about displaying images. It is OK to leave out basic support packages like os or glob.)_

**Credit:**
_E.g., "Originally developed by" or "Based on notebooks developed by" and then people's names, including journal article or software release citations if appropriate._
Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

_Provide a light narrative about this notebook, e.g., "This notebook will teach the user..."._

_Cite or link to any external information or documentation, and cross-reference to other notebooks._

### 1.1 Package Imports

_All package imports should be done in the first code cell._

_Provide explanation or external links to package documentation, where appropriate._

_E.g., Numpy is a fundamental package for scientific computing with arrays in Python (<a href="https://numpy.org">numpy.org</a>)._

_Use code cell comments to describe the packages being imported._

In [3]:
# general python packages
import time
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
import pandas
pandas.set_option('display.max_rows', 1000)

# LSST package for TAP queries
from lsst.rsp import get_tap_service, retrieve_query

# LSST package for Butler queries
import lsst.daf.butler as Butler

# LSST package for image display
import lsst.afw.display as afwDisplay
import lsst.geom

from astropy import units as u
from astropy.coordinates import SkyCoord

import matplotlib.pyplot as plt
import gc

# Astropy imports
from astropy.wcs import WCS
from astropy.visualization import make_lupton_rgb

import lsst.geom as geom
# Object for multi-band exposures
from lsst.afw.image import MultibandExposure

afwDisplay.setDefaultBackend('matplotlib')

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)

plt.style.use('tableau-colorblind10')
%matplotlib inline

### 1.2 Define Functions and Parameters

_If your notebook defines functions or parameters to use later or throughout, do it here in sub-section 1.2._

_It is OK to rename the subsection to be more specific to the notebook, and/or to use sub-sub-sections like "1.2.1 Define global cosmological parameter values" or "1.2.2 Define a function to make an image cutout"._

_It is OK to remove this sub-section if it is not being used._

## 2. Search data for a supernova for which to create image cutouts

_Use numbers for sections, sub-sections, and sub-sub-sections to enable referencing, e.g., "I'm having trouble with the second code cell in Section 2.3."_

_Use section titles that actively describe what is being done, e.g., "Create a color-magnitude diagram" instead of "Plot", so that the auto-generated table of contents is easy to navigate._

### 2.1 Section Sub-heading

#### 2.1.1 Section Sub-sub-heading

In [4]:
service = get_tap_service()
assert service is not None

In [5]:
# galaxy cluster coords: 55.6521739130, -31.9834710744
# grab a DiaObject identified in Tutorial 07a to be a candidate Type 1a SN with relatively few number of 
# nDiaSources (frames over which flux varies): RA=51.2332445 Dec=-44.509536 DiaObjID=1248649384967537762
#%%time

# First setup the search like in tutorial 07a:
redshift_min = 0.1
redshift_max = 0.3
snia_peak_mag = -19.0
snia_peak_mag_range = 0.5
snia_peak_mg_max = 24.0
snia_peak_mi_max = 24.0
snia_ampl_mr_min = 1.5
snia_ampl_mr_max = 5.5
snia_peak_mr_min = 18.82
snia_peak_mr_max = 22.46
# pick a set on the smaller end of time to make the output cutouts managable:
snia_nDiaSources_min = 15
snia_nDiaSources_max = 20

# Print the contents of the catalog DiaSource or DiaObject so you see what you can access
#results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
#                         "WHERE table_name = 'dp02_dc2_catalogs.DiaObject'")
#                         "WHERE table_name = 'dp02_dc2_catalogs.DiaSource'")
#results.to_table().to_pandas()
#according to the DPDD, the DiaSource tables should have the ccdVisitId
#results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
#                         "WHERE table_name = 'dp02_dc2_catalogs.Object'")
#results.to_table().to_pandas()


In [6]:
%%time

# retrieve candidate SN:
results = service.search("SELECT ra, decl, diaObjectId, nDiaSources, "
                         "scisql_nanojanskyToAbMag(rPSFluxMin) AS rMagMax, "
                         "scisql_nanojanskyToAbMag(rPSFluxMax) AS rMagMin, "
                         "scisql_nanojanskyToAbMag(gPSFluxMax) AS gMagMin, "
                         "scisql_nanojanskyToAbMag(iPSFluxMax) AS iMagMin, "
                         "scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) AS rMagAmp "
                         "FROM dp02_dc2_catalogs.DiaObject "
                         "WHERE nDiaSources > "+str(snia_nDiaSources_min)+" "
                         "AND nDiaSources < "+str(snia_nDiaSources_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_peak_mr_min)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_peak_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(gPSFluxMax) < "+str(snia_peak_mg_max)+" "
                         "AND scisql_nanojanskyToAbMag(iPSFluxMax) < "+str(snia_peak_mi_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_ampl_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_ampl_mr_min)+" ",
                         maxrec=1000)

#to use visitinfo you have to add to end of query:
#"ON visinfo.ccdVisitId = src.ccdVisitId "\
#        "WHERE src.objectId = "+str(sel_objid)+" "

# Can you get visit # added to this?

DiaObjs = results.to_table()
del results

CPU times: user 49.1 ms, sys: 61 µs, total: 49.2 ms
Wall time: 8.54 s


In [16]:
#display the table
#DiaObjs

In [8]:
# Initialize the Butler  
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = Butler.Butler(config, collections=collection)
registry = butler.registry

In [9]:
# look again at DiaSource catalog contents
results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
                         "WHERE table_name = 'dp02_dc2_catalogs.DiaSource'")
results.to_table().to_pandas()

,column_name,datatype,description,unit
0,apFlux,double,Flux within 12.0-pixel aperture,nJy
1,apFlux_flag,boolean,General Failure Flag,
2,apFlux_flag_apertureTruncated,boolean,Aperture did not fit within measurement image,
3,apFluxErr,double,Flux uncertainty within 12.0-pixel aperture,nJy
4,bboxSize,long,Bounding box of diaSource footprint,
5,ccdVisitId,long,Unique ID of CCD and visit where this source w...,
6,centroid_flag,boolean,"General failure flag, set if anything went wrong",
7,centroid_neg_flag,boolean,"Failure flag for negative, set if anything wen...",
8,centroid_pos_flag,boolean,"Failure flag for positive, set if anything wen...",
9,coord_dec,double,Fiducial ICRS Declination of centroid used for...,deg


In [10]:
query = "SELECT TOP " + str(10) + " " + \
        "diaObjectId, ra, decl, ccdVisitId " + \
        "FROM dp02_dc2_catalogs.DiaSource " #+ \

results = service.search(query)
results.to_table().to_pandas()

,diaObjectId,ra,decl,ccdVisitId
0,1249520198176735305,54.930877,-44.575409,1155625065
1,1249520198176735305,54.930899,-44.575532,300233003
2,1249520198176735305,54.930897,-44.575497,1192238175
3,1249520198176735305,54.930848,-44.575457,1190816014
4,1249520198176735305,54.930854,-44.575437,769374016
5,1249520198176735305,54.930858,-44.575459,1192195048
6,1249520198176735305,54.930868,-44.575486,212700025
7,1249520198176735305,54.930887,-44.575504,1193690021
8,1249520198176735305,54.930925,-44.575498,963954016
9,1249520198176735305,54.930912,-44.575480,941523080


In [11]:
# just pick a source to use for the cutouts, like the first one
ra, dec = DiaObjs['ra'][0], DiaObjs['decl'][0]
sel_objid = DiaObjs['diaObjectId'][0]
print(sel_objid)

center_coords = SkyCoord(ra, dec, frame='icrs', unit='deg')
search_radius = 1.0*u.arcmin
use_center_coords = str(ra)+", "+str(dec) #"62, -37"
use_radius = "0.00028" #1 arcsec in deg

print(center_coords)
print(search_radius)
                
# find the source using a small search radius, but there are too many DiaSources in a small area (weird)
query = "SELECT TOP " + str(20) + " " + \
        "diaObjectId, ra, decl, coord_ra, coord_dec, ccdVisitId " + \
        "FROM dp02_dc2_catalogs.DiaSource " + \
        "WHERE CONTAINS(POINT('ICRS', ra, decl), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " 
        
visits = service.search(query)
visits.to_table().to_pandas()

1252537258083352596
<SkyCoord (ICRS): (ra, dec) in deg
    (69.0475206, -44.512438)>
1.0 arcmin


,diaObjectId,ra,decl,coord_ra,coord_dec,ccdVisitId
0,1252537258083352618,69.047894,-44.512513,69.047890,-44.512493,1225448055
1,1252537258083352618,69.047792,-44.512513,69.047890,-44.512493,1207479042
2,1252537258083352618,69.047835,-44.512535,69.047890,-44.512493,1225449032
3,1252537258083352618,69.047857,-44.512454,69.047890,-44.512493,1029869044
4,1252537258083352618,69.047825,-44.512508,69.047890,-44.512493,1185125094
5,1252537258083352618,69.047814,-44.512446,69.047890,-44.512493,724492113
6,1252537258083352618,69.047758,-44.512469,69.047890,-44.512493,971987097
7,1252537258083352618,69.047824,-44.512457,69.047890,-44.512493,971923151
8,1252537258083352618,69.047778,-44.512491,69.047890,-44.512493,243089152
9,1252537258083352618,69.047896,-44.512432,69.047890,-44.512493,1028603025


In [14]:
# find the source by matching the source ID, which gives you the exact source
query = "SELECT TOP " + str(100) + " " + \
        "diaObjectId, ra, decl, coord_ra, coord_dec, ccdVisitId " + \
        "FROM dp02_dc2_catalogs.DiaSource " + \
        "WHERE diaObjectId = " + str(sel_objid) + " "

visits = service.search(query)
visits.to_table().to_pandas()

,diaObjectId,ra,decl,coord_ra,coord_dec,ccdVisitId
0,1252537258083352596,69.047453,-44.512402,69.047521,-44.512438,471436047
1,1252537258083352596,69.047519,-44.512372,69.047521,-44.512438,1180104055
2,1252537258083352596,69.047458,-44.512448,69.047521,-44.512438,1213299152
3,1252537258083352596,69.047513,-44.512452,69.047521,-44.512438,7906133
4,1252537258083352596,69.047547,-44.512452,69.047521,-44.512438,1010501185
5,1252537258083352596,69.047467,-44.512383,69.047521,-44.512438,646785018
6,1252537258083352596,69.047645,-44.512436,69.047521,-44.512438,1039028125
7,1252537258083352596,69.047594,-44.512434,69.047521,-44.512438,904692001
8,1252537258083352596,69.047594,-44.512395,69.047521,-44.512438,669866097
9,1252537258083352596,69.047531,-44.512457,69.047521,-44.512438,1184910084


In [13]:
# borrow this function from notebook tutorial 03a
# Crap this does NOT support calexp, ugh.
def cutout_coadd(butler, ra, dec, datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.
    
    CCW snagged this routine from tutorial notebook 03a

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Servant providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    #coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}
    print(dataId, parameters)
    cutout_image = butler.get(datasetType, #parameters=parameters,
                              dataId=dataId)

    return cutout_image


In [15]:
# now try to make the cutout using butler
print(visits['ccdVisitId'][0])
datasetType = 'calexp'
# this test works:
#dataId = {'visit': 192350}

# WE ARE STUCK HERE: ccdVisitId apparently does not exist in the database of images, yet there it is in the diaobject
dataId = {'visit': visits['ccdVisitId'][0]}
datasetRefs = set(registry.queryDatasets(datasetType, dataId=dataId))

#for i, ref in enumerate(datasetRefs):
#    calexp = butler.getDirect(ref)
#    print(i, ' calexp.detector.getId(): ', calexp.detector.getId())
#    if i > 2:
#        print('...')
#        break

        
for i, ref in enumerate(datasetRefs):
    calexp = butler.getDirect(ref)
    print(i, ' calexp.detector.getId(): ', calexp.detector.getId())
    fig = plt.figure()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(calexp.image)
    plt.show()
    if i > 2:
        print('...')
        break


471436047


DataIdValueError: Could not fetch record for required dimension visit via keys {'instrument': 'LSSTCam-imSim', 'visit': 471436047}.

In [81]:
# the following is testing junk

# now try to make the cutout using butler
radec = lsst.geom.SpherePoint(ra*lsst.geom.degrees,dec*lsst.geom.degrees)

skymap = butler.get('skyMap')
#cutout = cutout_coadd(butler, ra, dec, band='g', datasetType='deepCoadd', cutoutSideLength=1000)

# To get the visits used to contruct the deepCoadd
#unique_visits = list(set([i['visit'] for i in cutout.getInfo().getCoaddInputs().ccds]))


# Look up the tract, patch for the RA, Dec 
# Looks like tract and patch are NOT identifiers used for calexp. Must be DeepCoadd only
tractInfo = skymap.findTract(radec)
my_patch = tractInfo.findPatch(radec)

# note you can't specify both band and visit, because a visit only observes in 1 band
#dataId = {'visit': visits['ccdVisitId'][0], 'detector': 175}#, 'tract': tractInfo, 'patch': my_patch}

# The above has some error related to the visit #, so try test obs from tutorial 04b:
dataId = {'visit': 192350, 'detector': 175}
print(butler.registry.getDatasetType('calexp'))


# Select a position at roughly the center of the galaxy cluster:
cutout_image = cutout_coadd(butler, ra, dec, datasetType='calexp',
                            cutoutSideLength=50, dataId=dataId)

print(cutout_image.image)
#fig, ax = plt.subplots()
#display = afwDisplay.Display(frame=fig)
#display.scale('asinh', 'zscale')
#display.mtv(cutout_image.image)
#plt.imshow(cutout_image.image)
#plt.show()
#remove_figure(fig)
np.shape(cutout_image.image)

# deep coadd, we dont' want this because we want the individual time slices
#my_deepCoadd = butler.get('deepCoadd', dataId=dataId)
# datasetType of calexp = processed visit images, we want this for time series
#exposures = butler.get('calexp', dataId=dataId)

#xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
#bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
#patch = tractInfo.getSequentialPatchIndex(patchInfo)

#coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
#parameters = {'bbox': bbox}

#cutout_image = butler.get(datasetType, parameters=parameters,
#                           dataId=coaddId)

DatasetType('calexp', {band, instrument, detector, physical_filter, visit_system, visit}, ExposureF)
{'visit': 192350, 'detector': 175} {'bbox': Box2I(corner=Point2I(9022, 2708), dimensions=Extent2I(50, 50))}
[[ 173.24623   109.75976  -118.22696  ...  -43.87422    11.406144
    21.049547]
 [ -23.781313  220.54593    -8.997193 ...  -65.52727   -20.900955
   -15.608138]
 [ -50.302032  -37.39472    50.83145  ...  -44.288467   13.046585
    62.593796]
 ...
 [-136.2568    145.401      37.3945   ...   19.183275   42.69411
    -3.638317]
 [  24.348206   48.676422  -59.46411  ...  -58.0684     62.093555
    77.05068 ]
 [ 107.73544    60.37006   127.37406  ...  120.352036   89.114334
   108.9738  ]], bbox=(minimum=(0, 0), maximum=(4071, 3999))


()

In [83]:
cutout_image.image[1]

TypeError: cannot unpack non-iterable int object